In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import sklearn
from tqdm import tqdm
import skimage
from skimage import transform
from numpy import load
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, Concatenate, Input
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
drive.mount('/content/drive')

In [ ]:
# !mkdir "/content/Data"
# !unzip "/content/drive/Shareddrives/Multiverse Clones/Hand Gestures.zip" -d "/content/Data"

In [ ]:
# img_float32 = np.float32(x_train[0])
# gryimg = cv2.cvtColor(img_float32, cv2.COLOR_BGR2GRAY)

# CONVERTING IMAGE FILES TO ARRAY AND ASSIGNING LABELS
# imageSize = 50
def get_data(folder,countMax):
    x = []
    y = []
    for folderName in os.listdir(folder):
        # if folderName in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']:
        #     label = ord(folderName) - 65
        # elif folderName in ['del']:
        #     label = 26
        # elif folderName in ['nothing']:
        #     label = 27
        # elif folderName in ['space']:
        #     label = 28
        # else:
        #     label = 29
        label = int(folderName[0:2])
        print(folderName , label)
        count = 0
        for image_filename in tqdm(os.listdir(folder + "/" + folderName)[0:countMax]):
            img = cv2.imread(folder +'/' + folderName + '/' + image_filename)
            if img is not None:
                # img_small_size = skimage.transform.resize(img, (imageSize, imageSize, 3))
                # x.append(np.array(img_small_size))
                img = cv2.Canny(img, 100, 200)
                x.append(img)
                y.append(label)
    x = np.array(x)
    y = np.array(y)
    return x, y

def saveNpy(name,array):
  np.save(name,array)

def loadNpy(location):
  return load(location)
  # load('/content/drive/Shareddrives/Multiverse Clones/SelfMadeData/NpyFiles/data_train_attribute.npy')

def cropImage(a,b):
  for i in range(0,b):
    yb = 0
    yt = a.shape[1]
    for i in range(0,len(a)):
      for j in range(0,len(a[i])):
        if(a[i][j]!=0):
          yt = min(yt,i)
          yb = max(yb,i)
          break

    xl = a.shape[1]
    xr = 0
    for j in range(0,len(a[0])):
      for i in range(0,len(a)):
        if(a[i][j]!=0):
          xl = min(xl,j)
          xr = max(xr,j)
          break
    a = a[yt:yb,xl:xr]
  return a

def showGrayImage(img):
  plt.imshow(img,cmap = 'gray')
  plt.show()

def resizeImage(img,imageSize):
  return skimage.transform.resize(img, (imageSize, imageSize))

def oneHotEncoding(a,size):
  return tf.one_hot(a.reshape(-1), size)

def getOutline(img):
  outline_kernel = np.array([[-1, -1, -1],
                           [-1, 8, -1],
                           [-1, -1, -1]])
  return cv2.filter2D(img, -1, outline_kernel)

def convertImagesToOutline(a):
  b = []
  for i in a:
    tmp = getOutline(i)
    b.append(tmp)
  return np.array(b)

def cropAndResize(a):
  b = []
  for i in tqdm(a):
    b.append(resizeImage(cropImage(i,3),50))
  return np.array(b)

def take_photo(filename='photo.jpg', quality=1):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


In [ ]:
# x, y = get_data("/content/Data/",200)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 2)
# del x
# del y

In [ ]:
# x_train = cropAndResize(x_train)
# x_test = cropAndResize(x_test)

In [ ]:
# saveNpy('x_train.npy',x_train)
# saveNpy('x_test.npy',x_test)
# saveNpy('y_train.npy',y_train)
# saveNpy('y_test.npy',y_test)

In [ ]:
x_train = load('/content/drive/Shareddrives/Multiverse Clones/WorkingModelData/Npy of 200pg/x_train.npy')
x_test = load('/content/drive/Shareddrives/Multiverse Clones/WorkingModelData/Npy of 200pg/x_test.npy')
y_train = load('/content/drive/Shareddrives/Multiverse Clones/WorkingModelData/Npy of 200pg/y_train.npy')
y_test = load('/content/drive/Shareddrives/Multiverse Clones/WorkingModelData/Npy of 200pg/y_test.npy')

In [ ]:
y_train = oneHotEncoding(y_train,11)
y_test = oneHotEncoding(y_test,11)

In [ ]:
rnd_idx = random.randint(0,len(x_train)-1)
img = x_train[rnd_idx]
label = y_train[rnd_idx]

showGrayImage(img)
print(label)

In [ ]:
# x_train = convertImagesToOutline(x_train)
# x_test = convertImagesToOutline(x_test)

In [ ]:
# Set input shape
sample_shape = x_train[0].shape
img_width, img_height = sample_shape[0], sample_shape[1]
input_shape = (img_width, img_height, 1)

# Reshape data 
x_train_model = x_train.reshape(len(x_train), input_shape[0], input_shape[1], input_shape[2])
x_test_model  = x_test.reshape(len(x_test), input_shape[0], input_shape[1], input_shape[2])

In [ ]:
print(input_shape)
print(x_train.shape)
print(x_train_model.shape)

In [ ]:
model = Sequential(name="HandSignToSpeech")

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.15))
# model.add(Dense(128, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x=x_train_model, y=y_train, epochs=10)

In [ ]:
model.evaluate(x=x_test_model, y=y_test)

In [ ]:
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  img = cv2.imread('/content/photo.jpg')
  plt.imshow(img)
  plt.show()
except Exception as err:
  print(str(err))

In [ ]:
img = cv2.imread('/content/photo.jpg')
img = cv2.Canny(img, 0, 250)
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
img = cv2.flip(img, 1)
plt.imshow(img,cmap = 'gray')

In [ ]:
img = img[10:350,250:490]
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
imageSize = 50
img = skimage.transform.resize(img, (imageSize, imageSize))
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
pred = model.predict(img.reshape(1, img.shape[0], img.shape[1], 1))
pred_cls = np.argmax(pred)
print(pred,pred_cls)